# Wikipedia - this day in history <small>(step 2 - get link data)</small>
---
**Goal:** get link data (i.e. page data) for links in wikipedia_tdih.db that have no data in the database or have outdated data.

**Notes:**  
- this notebook is for step two out of three of this project. 
- the notebook for the first step is [TADS_wikipedia_tdih_main_step_01_get_day_data_30jan21](https://github.com/Bianca-Aguglia/TADS_wikipedia_this_day_in_history_build_dataset/tree/master/notebooks)
- the notebook for the third step is [TADS_wikipedia_tdih_main_step_03_get_image_data_10feb21](https://github.com/Bianca-Aguglia/TADS_wikipedia_this_day_in_history_build_dataset/tree/master/notebooks) 

### Process flow: <small>(with checkmarks for steps done in this notebook)</small>
- get day data
    - get day data from wikipedia API, process it, and save it to wikipedia_tdih.db
- get link data
    - [x] query wikipedia_tdih.db for:
        - [x] links that are new (i.e. in wiki_link but not in wiki_get_link_data_log)
        - [x] links that have have not been updated since a specified_date
    - [x] get link data from wikipedia API and extract the following:
        - [x] page_size
        - [x] incoming_links
        - [x] coordinates
        - [x] page_score
        - [x] first_paragraphs
        - [x] image_url
        - [x] image_file
        - [x] wikibase_shortdesc
        - [x] wikibase_item
        - [x] wiki_desc
        - [x] page_views
    - [x] update wikipedia_tdih.db tables
        - [x] wiki_link_data_log
        - [x] wiki_link_info
        - [x] wiki_link_page_views
        - [x] wiki_image <small>*</small>
        - [x] wiki_image_usage <small>*</small>
- get image data
    - get image data from wikipedia API, process it, and save it to wikipedia_tdih.db  
  
<small>* wiki_image and wiki_image_usage are updated in this step (step two) only with data that shows relationship between wiki_link and wiki_image. The image data from Wikipedia is retrieved, processed, and saved to the database in step three.</small>

In [3]:
import sqlite3
import requests
import time
import random
import config
import datetime as dt

In [4]:
DATABASE_FILE = config.DATABASE_FILE
DOE = config.DOE
URL = config.WIKIPEDIA_URL
HEADERS = config.HEADERS
PARAMS = {'titles': '', # placeholder for page
          'action': 'query',
          'format': 'json',
          'prop': 'cirrusbuilddoc|cirruscompsuggestbuilddoc|extracts|pageimages|pageprops|pageterms|pageviews',
          'piprop': 'original',
          'exintro': True # gives the first paragraphs of a page (before detail sections)
         }
# explanation of values for 'prop'
# cirrusbuilddoc for text_bytes (aka page_size), incoming_link, coordinates (if any)
# cirruscompsuggestbuilddoc for score
# extracts (+ exintro) for first_paragraph(s) (regex to clean up) (might me better to use ['cirrusbuilddoc']['text_source'])
# pageimages for pageimage (with piprop='original') for full size image (but doesn't return image file anymore)
# pageprops for page_image_free, wikibase_short_description, wikibase_item
# pageterms for description (similar but not identital to wikibase_short_description)
# pageviews for pageviews :-)

In [22]:
def link_data_main(database_file = DATABASE_FILE, date = '', batch_size = 5, doe = DOE, print_progress = 0,
                   sleep_range = (5,10), url = URL, params = PARAMS, headers = HEADERS):
    """
    Main function for getting and processing the link data for a group of links in wikipedia_tdih.db.
    It uses several helper functions to break down the process into simple steps:
        - get_links_to_update_from_db
        - get_link_data
        - extract_link_data
        - add_link_data_to_db
    
    Params:
        database_file: database file to save the data to
        date: default of '' indicates that only links without data should be selected
              if date is given, select the links that have data but data has not been updated since specified date
              if date is given, format should be '%Y-%m-%d' (e.g. '2021-01-02' for January 2nd, 2021)
        batch_size: no. of links to request data for in a single call to wikipedia API (to reduce no. of API calls)
        doe: date of entry (defaults to current day)
        print_progress: if different than 0, prints batch number being processed (every print_progress no. of batches)
        sleep_range: tuple of integers indicating the lower_limit and upper_limit of how long to wait between
                     API calls
        url: url to use for request to wikipedia API
        params: params to use in request to wikipedia API
        headers: headers to use in request to wikipedia API
        
    Returns:
        None
    """
    with sqlite3.connect(DATABASE_FILE) as connection:
        cursor = connection.cursor()
    
        # select from wikipedia_tdih.db the links we need data for
        links_list = get_links_to_update_from_db(cursor, date)
        links_list = links_list[:11] # keep this here only while testing
    
        # exit if no links need to have data retrieved or updated
        if not links_list:
            return 'no link data needs to be retrieved / updated'
        
        # keep track of failed_request (break if failed_requests > 5)
        failed_request = 0
    
        # process links in batches of size batch_size
        for i in range(0, len(links_list), batch_size):
            # slice links_list into batches
            link_batch = links_list[slice(i, i + batch_size)]
            
            # if print_progress is given
            if print_progress:
                if i % print_progress:
                    print(f'processing batch {i+1} of {len(links_list)}')
                       
            # get link data for links in batch
            resp = get_link_data(link_batch, url = url, params = params, headers = headers)
            
            # extract link_data (if resp.status_code == requests.codes.ok)
            if resp.status_code == requests.codes.ok:
                page_dicts = extract_link_data(resp)

            else:
                # update wiki_link_data_log with response status_code
#                 update_table_wiki_link_data_log(link_id, status_code, cursor, doe = DOE)
                update_table_wiki_link_data_log(link_batch, resp.status_code)
                failed_request += 1
                if failed_request % 5 == 0:
                    return f'failed requests: {failed_request}'
                continue

            # add link_data to db
            for page_dict in page_dicts:
                add_link_data_to_db()
                
            # sleep time before the next API call
            time.sleep(round(random.uniform(sleep_range[0], sleep_range[1]), 2))

In [18]:
def get_links_to_update_from_db(database_cursor, date = ''):
    """
    Select from wikipedia_tdih.db the links for which wikipedia data is needed.
    
    There are two cases in which data is needed for a specific link:
        1. the link has just been added to the wikipedia_tdih.db and link data has not been requested from 
           wikipedia API yet
        2. the existing data for the link needs to be updated
    
    Params:
        date: default of '' indicates that only links without data should be selected
              if date is given, select the links that have data but data has not been updated since specified date
              if date is given, format should be '%Y-%m-%d' (e.g. '2021-01-02' for January 2nd, 2021)
              
    Returns:
        links_list: list of links for which wikpedia data is needed
    """
    
    c = database_cursor
    
    # if no date is given
    if not date:
        # select the links that are in wiki_link but not in wiki_get_link_data_log
        # (these are links which have just been added to wiki_link)
        links_list = c.execute('''SELECT link_id, link_url FROM wiki_link WHERE link_id NOT IN (
                                    SELECT link_id FROM wiki_link_data_log) ''').fetchall()
    
    # if date is given
    else:
        links_list = c.execute('''SELECT link_id, link_url FROM wiki_link WHERE link_id IN (
                                    SELECT link_id FROM wiki_get_link_data_log WHERE doe > ?)''', (date,)).fetchall()
      
    return links_list

In [32]:
def get_link_data(link_batch, url = URL, params = PARAMS, headers = HEADERS):
    """
    Get link data from Wikipedia API. Links are processed in batches (usually of size 5) to reduce the number
    of API calls.
    
    Params:
        link_batch: list of links to get data for (usually a batch of size 5)
        url: url for wikipedia API
        params: params to use in API call
    """
    # join the urls for the links in the list link_batch and assign them to request params
    # wikipedia uses these urls as titles
    # each link is a tuple in format (link_id, link_url)
    # link url needs to be stripped of '/wiki/'
    # e.g. link url '/wiki/Albert_Einstein' needs to be changed to 'Albert_Einstein'
    titles = '|'.join(wiki_link[1].replace('/wiki/', '') for wiki_link in link_batch)
    print(titles)
    params['titles'] = titles
    
    # request data
    resp = requests.get(url = url, headers = headers, params = params)
    
    return resp

In [30]:
def extract_link_data(response):
    """
    Extract link data
    
    Params:
        response: response data received from Wikipedia API
        
    Returns:
        page_dicts: list of dictionaries with page data
    
    """
    resp = response.json()
    status_code = response.status_code
    page_dicts = []
    
    # get the wiki_links from the response (to match them to wiki_link in database)
    wiki_links_list = resp['query']['normalized']
    wiki_links_dict = {link['to']: link['from'] for link in wiki_links_list}
    
    print(wiki_links_list)
    print(wiki_links_dict)
    print(resp['query']['pages'].keys())
    for page_id in resp['query']['pages'].keys():
        print(resp['query']['pages'][page_id]['title'])
    
    for page_id in resp['query']['pages'].keys():
        page_dict = resp['query']['pages'][page_id]
        
        # wiki_link is normalized by Wikipedia (e.g. 'Albert_Einstein' normalized to 'Albert Einstein')
        # extract it and find its original value
        wiki_link_normalized = page_dict['title']
        print(wiki_link_normalized)
        wiki_link = wiki_links_dict.get('wiki_link_normalized', wiki_link_normalized)
#         wiki_link = wiki_links_dict[wiki_link_normalized]
        
        # build up the data dict
        data_dict = {'wiki_link': f'/wiki/{wiki_link}', # change this so links saved to db are stripped of '/wiki/'?
                     'status_code': status_code,
                     'page_size': page_dict['cirrusbuilddoc']['text_bytes'],
                     'incoming_links': page_dict['cirrusbuilddoc']['incoming_links'],
                     'coordinates': page_dict['cirrusbuilddoc']['coordinates'],
                     'page_score': page_dict['cirruscompsuggestbuilddoc'][f'{page_id}t']['score_explanation']['value'],
                     'first_paragraphs': BeautifulSoup(page_dict['extract']).text.strip(),
                     'image_url': page_dict['original']['source'],
                     'image_file': page_dict['pageprops']['page_image_free'],
                     'wikibase_shortdesc': page_dict['pageprops']['wikibase-shortdesc'],
                     'wikibase_item': page_dict['pageprops']['wikibase_item'],
                     'wiki_desc': page_dict['terms']['description'],
                     'page_views': page_dict['pageviews']}
        page_dicts.append(data_dict)
        
    return page_dicts

In [10]:
def add_link_data_to_db(page_data_dict, db = DATABASE_FILE, doe = DOE):
    """
    Add wiki_link data to wikipedia_tdih.db
    """
    # connect to wikipedia_tdih.db
    with sqlite3.connect(db) as conn:
        c = conn.cursor()
    
        # get link_id from db
        c.execute('SELECT link_id, link_url FROM wiki_link WHERE link_url = ?', (page_data_dict[wiki_link],))
        link_id = c.fetchone()[0]
    
        # update wiki_link tables
        update_table_wiki_link_data_log(link_id, page_data_dict['status_code'], c, doe)
        update_table_wiki_link_page_views(link_id, page_data_dict['page_views'], c, doe)
        update_table_wiki_link_info(link_id, page_data_dict, c, doe)
        
        # update wiki_image_tables
        image_id = update_table_wiki_image(page_data_dict['image_url'], page_data_dict['image_file'], c, doe)
        update_table_wiki_image_usage(image_id, link_id, c, doe)  

In [11]:
def update_table_wiki_link_data_log(link_id, status_code, cursor, doe = DOE):
    """
    Update table wiki_link_data_log
    """
    
    data = (doe, link_id, status_code)
    
    cursor.execute('INSERT INTO wiki_link_data_log VALUES (null,?,?,?)', data)

In [12]:
def update_table_wiki_link_page_views(link_id, page_views, cursor, doe = DOE):
    """
    Update table wiki_link_size
    
    Params:
        link_id: id of wiki_link to update page_views for
        page_views: dictionary of page view data to update
        doe: date of entry
    """
    data = [(doe, link_id, date, views) for date, views in page_views.items()]
    
    cursor.executemany('INSERT INTO wiki_link_size VALUES (null,?,?,?)', data)

In [13]:
def update_table_wiki_link_info(link_id, page_dict, cursor, doe = DOE):
    """
    Update table wiki_link_size
    
    Params:
        link_id: id of wiki_link to update page_views for
        page_data_dict: dictionary with data retrieved from Wikipedia API
                        eg. page_dict = {'wiki_link': wiki_link,
                                         'status_code': status_code,
                                         'page_size': page_size,
                                         'incoming_links': incoming_links,
                                         'coordinates': coordinates,
                                         'page_score': page_score,
                                         'first_paragraphs': first_paragraphs,
                                         'image_url': image_url,
                                         'image_file': image_file,
                                         'wikibase_short_desc': wikibase_short_desc,
                                         'wikibase_item': wikibase_item,
                                         'wiki_desc': wiki_desc,
                                         'page_views': page_views}
        cursor: database cursor
        doe: date of entry
        
    Returns: None
    """
    # build up data tuple
    data = (doe, 
            link_id,
            page_dict['page_size'],
            page_dict['incoming_links'], 
            page_dict['coordinates'], 
            page_dict['page_score'],
            page_dict['first_paragraphs'], 
            page_dict['wikibase_short_description'],
            page_dict['wikibase_item'],
            page_dict['wiki_desc'])
    
    # insert data into wiki_link_info
    cursor.execute('INSERT INTO wiki_link_info VALUES (null,?,?,?,?,?,?,?,?,?,?)', data)

In [14]:
def update_table_wiki_image(image_url, image_file, cursor, doe = DOE):
    """
    Update table wiki_image if image is not already in the table.
    
    Params:
        image_url: wikipedia url for image used for wiki_link
        image_file: wikipedia file name for image used for wiki_link
        cursor: database cursor
        doe: date of entry (defaults to current day)
        
    Returns:
        image_id
    """
    # insert image_url if not already in wiki_image    
    cursor.execute('INSERT OR IGNORE INTO wiki_image VALUES (null,?,?)', (doe, image_url, image_file))
    
    # get the image_id (to update wiki_image_usage)
    cursor.execute('SELECT image_id FROM wiki_image WHERE image_url = ?', (image_url,))
    image_id = c.fetchone()[0]
    
    # return image_id
    return image_id

In [16]:
def update_table_wiki_image_usage(image_id, link_id, cursor, doe = DOE):
    """
    Update table wiki_image_usage.
    
    Params:
        image_id: id of image used by wiki_link (primary key in wiki_image)
        link_id: id of wiki_link (primary key in wiki_link)
        
    Returns:
        None
    """
    # insert data into wiki_image_usage
    cursor.execute('INSERT INTO wiki_image_usage VALUES (null,?,?,?)', (doe, image_id, link_id))

In [33]:
link_data_main(database_file = DATABASE_FILE, date = '', batch_size = 5, doe = DOE, print_progress = 1,
               sleep_range = (5,10), url = URL, params = PARAMS, headers = HEADERS)

%C3%87a%C4%9Fda%C5%9F_Atan|%C3%89va_Sz%C3%A9kely|Abel_Tasman|Academy_Awards|Adam_Sinclair
[{'from': 'Abel_Tasman', 'to': 'Abel Tasman'}, {'from': 'Academy_Awards', 'to': 'Academy Awards'}, {'from': 'Adam_Sinclair', 'to': 'Adam Sinclair'}]
{'Abel Tasman': 'Abel_Tasman', 'Academy Awards': 'Academy_Awards', 'Adam Sinclair': 'Adam_Sinclair'}
dict_keys(['-1', '-2', '1988', '324', '2267098'])
%C3%87a%C4%9Fda%C5%9F_Atan
%C3%89va_Sz%C3%A9kely
Abel Tasman
Academy Awards
Adam Sinclair
%C3%87a%C4%9Fda%C5%9F_Atan


KeyError: 'cirrusbuilddoc'